In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import time
import json

In [15]:
def bb_record():    
    now = dt.datetime.now()
    ny = now.strftime('%Y')
    nd = now.strftime('%Y%m%d')
    nm = now.strftime('%m')
    print(f'업데이트 시간:{now}')
    x = '4'
    result_lst = []
    for month in range(int(x.zfill(2)), int(nm)+1):
        url=f'https://sports.news.naver.com/kbaseball/schedule/index?date={nd}&month={month}&year={ny}&teamCode='
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        divs = soup.select('#calendarWrap>div')
        for div in divs:
            date=div.select('td')[0].text.strip()
            trs = div.select('tr')
            for tr in trs:
                tds = tr.select('td')
                if tr == trs[0]:
                    hour = tds[1].text.strip()
                    if len(tds) == 3:
                        break
                    elif tr.select('td.add_state'): 
                        away = tr.select('span.team_lft')[0].text
                        asc = tr.select('td>strong>em.vs')[0].text
                        home = tr.select('span.team_rgt')[0].text 
                        hsc = tr.select('span.suspended')[0].text
                        stadium = tds[4].text.strip()
                        bc = tds[3].text.strip()
                        win_pit = ''
                        lose_pit = ''
                        fin_bat = ''
                    elif tr.find_all('img')[2]['alt'] == "전력":
                        away = tr.select('span.team_lft')[0].text 
                        asc = ''
                        home = tr.select('span.team_rgt')[0].text
                        hsc = ''
                        stadium = tds[5].text.strip()
                        bc = tds[4].text.strip()
                        win_pit = ''
                        lose_pit = ''
                        fin_bat = ''
                    elif tr.find_all('img')[2]['alt'] == "경기영상":
                        away = tr.select('span.team_lft')[0].text 
                        asc = ''
                        home = tr.select('span.team_rgt')[0].text
                        hsc = ''
                        stadium = tds[4].text.strip()
                        bc = tds[3].text.strip()
                        win_pit = ''
                        lose_pit = ''
                        fin_bat = ''
                    else:
                        away = tr.select('span.team_lft')[0].text
                        asc = tr.select('strong.td_score')[0].text.split(':')[0]
                        home = tr.select('span.team_rgt')[0].text 
                        hsc = tr.select('strong.td_score')[0].text.split(':')[1]
                        stadium = tds[5].text.strip()
                        bc = tds[4].text.strip()
                        raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                        record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                        record_rp = requests.get(record_url)
                        record = json.loads(record_rp.text)
                        pit_record = record['result']['recordData']['pitchersBoxscore']
                        bat_record = record['result']['recordData']['etcRecords']
                        fin_bat_name = bat_record[0]['result'].split('(')[0]
                        bat_score_record = record['result']['recordData']['battersBoxscore']
                        win_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or \
                                        [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']
                        lose_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '패'] or \
                                        [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '패']
                        fin_bat_info = [bat_info for bat_info in list(bat_score_record.values())[2] if bat_info['name'] == fin_bat_name] or \
                                                                [bat_info for bat_info in list(bat_score_record.values())[3] if bat_info['name'] == fin_bat_name]
                        for wp in win_pit_info:
                            win_pit_name = wp['name'] + '(' + wp['wls'] + ')'
                            win_pit_inn = wp['inn']
                            win_pit_hit = str(wp['hit']).replace('0','무')
                            win_pit_ball = str(wp['bbhp']).replace('0','무')
                            win_pit_k = wp['kk']
                            win_pit_loss = str(wp['r']).replace('0','무')
                            win_pit_er = str(wp['er']).replace('0','무')
                            win_pit_nb = wp['bf']
                            win_pit_era = wp['era']
                            win_pit = {win_pit_name : [f'{win_pit_inn}이닝, {win_pit_hit}피안타, {win_pit_ball}사사구, {win_pit_k}탈삼진, {win_pit_loss}실점({win_pit_er}자책), 투구수{win_pit_nb}개, 평균자책{win_pit_era}']}
                        for lp in lose_pit_info:
                            lose_pit_name = lp['name'] + '(' + lp['wls'] + ')'
                            lose_pit_inn = lp['inn']
                            lose_pit_hit = str(lp['hit']).replace('0','무')
                            lose_pit_ball = str(lp['bbhp']).replace('0','무')
                            lose_pit_k = lp['kk']
                            lose_pit_loss = str(lp['r']).replace('0','무')
                            lose_pit_er = str(lp['er']).replace('0','무')
                            lose_pit_nb = lp['bf']
                            lose_pit_era = lp['era']
                            lose_pit = {lose_pit_name : [f'{lose_pit_inn}이닝, {lose_pit_hit}피안타, {lose_pit_ball}사사구, {lose_pit_k}탈삼진, {lose_pit_loss}실점({lose_pit_er}자책), 투구수 {lose_pit_nb}개, 평균자책 {lose_pit_era}']}
                        for fb in fin_bat_info:
                            fin_bat_nao = fin_bat_name + '(' + str(fb['batOrder']) +'번타자' + ')'
                            fin_bat_pa = fb['ab']
                            fin_bat_hit = fb['hit']
                            fin_bat_hr = fb['hr']
                            fin_bat_ball = fb['bb']
                            fin_bat_k = fb['kk']
                            fin_bat_hp = fb['rbi']
                            fin_bat_pt = fb['run']
                            fin_bat_avg = fb['hra']
                            fin_bat = {fin_bat_nao : [f'{fin_bat_pa}타수, {fin_bat_hit}안타({fin_bat_hr}홈런), {fin_bat_ball}볼넷, {fin_bat_k}삼진, {fin_bat_hp}타점, {fin_bat_pt}득점, 시즌타율 {fin_bat_avg}']}
                else:
                    hour = tds[0].text.strip()
                    if len(tds) == 3:
                        break
                    elif tr.select('td.add_state'):
                        away = tr.select('span.team_lft')[0].text 
                        asc = tr.select('td>strong>em.vs')[0].text
                        home = tr.select('span.team_rgt')[0].text
                        hsc = tr.select('span.suspended')[0].text
                        stadium = tds[3].text.strip()
                        bc = tds[2].text.strip()
                        win_pit = ''
                        lose_pit = ''
                        fin_bat = ''
                    elif tr.find_all('img')[2]['alt'] == "전력":
                        away = tr.select('span.team_lft')[0].text 
                        asc = ''
                        home = tr.select('span.team_rgt')[0].text
                        hsc = ''
                        stadium = tds[4].text.strip()
                        bc = tds[3].text.strip()
                        win_pit = ''
                        lose_pit = ''
                        fin_bat = ''
                    elif tr.find_all('img')[2]['alt'] == "경기영상":
                        away = tr.select('span.team_lft')[0].text 
                        asc = ''
                        home = tr.select('span.team_rgt')[0].text
                        hsc = ''
                        stadium = tds[4].text.strip()
                        bc = tds[3].text.strip()
                        win_pit = ''
                        lose_pit = ''
                        fin_bat = ''
                    else:
                        away = tr.select('span.team_lft')[0].text
                        asc = tr.select('strong.td_score')[0].text.split(':')[0]
                        home = tr.select('span.team_rgt')[0].text 
                        hsc = tr.select('strong.td_score')[0].text.split(':')[1]
                        stadium = tds[4].text.strip()
                        bc = tds[3].text.strip()
                        raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                        record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                        record_rp = requests.get(record_url)
                        record = json.loads(record_rp.text)
                        pit_record = record['result']['recordData']['pitchersBoxscore']
                        bat_record = record['result']['recordData']['etcRecords']
                        fin_bat_name = bat_record[0]['result'].split('(')[0]
                        bat_score_record = record['result']['recordData']['battersBoxscore']
                        win_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or \
                                        [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']
                        lose_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '패'] or \
                                        [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '패']
                        fin_bat_info = [bat_info for bat_info in list(bat_score_record.values())[2] if bat_info['name'] == fin_bat_name] or \
                                                                [bat_info for bat_info in list(bat_score_record.values())[3] if bat_info['name'] == fin_bat_name]
                        for wp in win_pit_info:
                            win_pit_name = wp['name'] + '(' + wp['wls'] + ')'
                            win_pit_inn = wp['inn']
                            win_pit_hit = str(wp['hit']).replace('0','무')
                            win_pit_ball = str(wp['bbhp']).replace('0','무')
                            win_pit_k = wp['kk']
                            win_pit_loss = str(wp['r']).replace('0','무')
                            win_pit_er = str(wp['er']).replace('0','무')
                            win_pit_nb = wp['bf']
                            win_pit_era = wp['era']
                            win_pit={win_pit_name : [f'{win_pit_inn}이닝, {win_pit_hit}피안타, {win_pit_ball}사사구, {win_pit_k}탈삼진, {win_pit_loss}실점({win_pit_er}자책), 투구수{win_pit_nb}개, 평균자책{win_pit_era}']}
                        for lp in lose_pit_info:
                            lose_pit_name = lp['name'] + '(' + lp['wls'] + ')'
                            lose_pit_inn = lp['inn']
                            lose_pit_hit = str(lp['hit']).replace('0','무')
                            lose_pit_ball = str(lp['bbhp']).replace('0','무')
                            lose_pit_k = lp['kk']
                            lose_pit_loss = str(lp['r']).replace('0','무')
                            lose_pit_er = str(lp['er']).replace('0','무')
                            lose_pit_nb = lp['bf']
                            lose_pit_era = lp['era']
                            lose_pit = {lose_pit_name : [f'{lose_pit_inn}이닝, {lose_pit_hit}피안타, {lose_pit_ball}사사구, {lose_pit_k}탈삼진, {lose_pit_loss}실점({lose_pit_er}자책), 투구수 {lose_pit_nb}개, 평균자책 {lose_pit_era}']}
                        for fb in fin_bat_info:
                            fin_bat_nao = fin_bat_name + '(' + str(fb['batOrder']) +'번타자' + ')'
                            fin_bat_pa = fb['ab']
                            fin_bat_hit = fb['hit']
                            fin_bat_hr = fb['hr']
                            fin_bat_ball = fb['bb']
                            fin_bat_k = fb['kk']
                            fin_bat_hp = fb['rbi']
                            fin_bat_pt = fb['run']
                            fin_bat_avg = fb['hra']
                            fin_bat = {fin_bat_nao : [f'{fin_bat_pa}타수, {fin_bat_hit}안타({fin_bat_hr}홈런), {fin_bat_ball}볼넷, {fin_bat_k}삼진, {fin_bat_hp}타점, {fin_bat_pt}득점, 시즌타율 {fin_bat_avg}']}
                                    
                result_lst.append([date, hour, away, asc, home, hsc, stadium, bc, win_pit, lose_pit, fin_bat])
    return result_lst
bb_record()

업데이트 시간:2023-08-03 19:00:00.840962


[['4.1 (토)',
  '14:00',
  '한화',
  '2',
  '키움',
  '3',
  '고척',
  'KBS2',
  {'문성현(승)': ['1이닝, 무피안타, 무사사구, 0탈삼진, 무실점(무자책), 투구수14개, 평균자책0.00']},
  {'장시환(패)': ['0 ⅔이닝, 3피안타, 1사사구, 0탈삼진, 1실점(1자책), 투구수 22개, 평균자책 13.50']},
  {'이형종(6번타자)': ['4타수, 1안타(0홈런), 0볼넷, 1삼진, 1타점, 0득점, 시즌타율 0.250']}],
 ['4.1 (토)',
  '14:00',
  'KIA',
  '1',
  'SSG',
  '4',
  '문학',
  'MBC',
  {'김광현(승)': ['5이닝, 4피안타, 3사사구, 3탈삼진, 1실점(1자책), 투구수87개, 평균자책1.80']},
  {'앤더슨(패)': ['6 ⅔이닝, 6피안타, 2사사구, 2탈삼진, 3실점(3자책), 투구수 91개, 평균자책 4.05']},
  {'박성한(7번타자)': ['4타수, 1안타(0홈런), 0볼넷, 0삼진, 2타점, 0득점, 시즌타율 0.250']}],
 ['4.1 (토)',
  '14:00',
  'LG',
  '6',
  'KT',
  '11',
  '수원',
  'MBC SPORTS+, SPOTVSPOTV2',
  {'벤자민(승)': ['6이닝, 2피안타, 무사사구, 4탈삼진, 1실점(무자책), 투구수87개, 평균자책0.00']},
  {'켈리(패)': ['5 ⅓이닝, 8피안타, 무사사구, 3탈삼진, 6실점(6자책), 투구수 85개, 평균자책 10.13']},
  {'알포드(3번타자)': ['4타수, 4안타(1홈런), 0볼넷, 0삼진, 3타점, 1득점, 시즌타율 1.000']}],
 ['4.1 (토)',
  '14:00',
  '롯데',
  '10',
  '두산',
  '12',
  '잠실',
  'SBS',
  {'최지강(승)': ['0 ⅔이닝, 무피안타, 무사사구, 1탈삼진, 무실점(무자책), 투구수9개

In [45]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
print(f'업데이트 시간:{now}')
df_bb=pd.DataFrame(result_lst, columns=['날짜', '시간', '원정팀', '원정팀득점', '홈팀', '홈팀득점', '구장', '중계방송사', '승리투수정보', '패전투수정보', '결승타자정보'])
df_bb

업데이트 시간:2023-08-03 09:01:00.953258


,날짜,시간,원정팀,원정팀득점,홈팀,홈팀득점,구장,중계방송사,승리투수정보,패전투수정보,결승타자정보
0,4.1 (토),14:00,한화,2,키움,3,고척,KBS2,"{'문성현(승)': ['1이닝, 무피안타, 무사사구, 0탈삼진, 무실점(무자책), 투구수14개, 평균자책0.00']}","{'장시환(패)': ['0 ⅔이닝, 3피안타, 1사사구, 0탈삼진, 1실점(1자책), 투구수 22개, 평균자책 13.50']}","{'이형종(6번타자)': ['4타수, 1안타(0홈런), 0볼넷, 1삼진, 1타점, 0득점, 시즌타율 0.250']}"
1,4.1 (토),14:00,KIA,1,SSG,4,문학,MBC,"{'김광현(승)': ['5이닝, 4피안타, 3사사구, 3탈삼진, 1실점(1자책), 투구수87개, 평균자책1.80']}","{'앤더슨(패)': ['6 ⅔이닝, 6피안타, 2사사구, 2탈삼진, 3실점(3자책), 투구수 91개, 평균자책 4.05']}","{'박성한(7번타자)': ['4타수, 1안타(0홈런), 0볼넷, 0삼진, 2타점, 0득점, 시즌타율 0.250']}"
2,4.1 (토),14:00,LG,6,KT,11,수원,"MBC SPORTS+, SPOTVSPOTV2","{'벤자민(승)': ['6이닝, 2피안타, 무사사구, 4탈삼진, 1실점(무자책), 투구수87개, 평균자책0.00']}","{'켈리(패)': ['5 ⅓이닝, 8피안타, 무사사구, 3탈삼진, 6실점(6자책), 투구수 85개, 평균자책 10.13']}","{'알포드(3번타자)': ['4타수, 4안타(1홈런), 0볼넷, 0삼진, 3타점, 1득점, 시즌타율 1.000']}"
3,4.1 (토),14:00,롯데,10,두산,12,잠실,SBS,"{'최지강(승)': ['0 ⅔이닝, 무피안타, 무사사구, 1탈삼진, 무실점(무자책), 투구수9개, 평균자책0.00']}","{'문경찬(패)': ['0이닝, 3피안타, 무사사구, 0탈삼진, 3실점(3자책), 투구수 7개, 평균자책 0.00']}","{'로하스(3번타자)': ['6타수, 2안타(1홈런), 0볼넷, 1삼진, 5타점, 2득점, 시즌타율 0.333']}"
4,4.1 (토),14:00,NC,8,삼성,0,대구,"KBS N SPORTS, SBS SPORTS","{'페디(승)': ['5이닝, 4피안타, 3사사구, 6탈삼진, 무실점(무자책), 투구수108개, 평균자책0.00']}","{'뷰캐넌(패)': ['5이닝, 8피안타, 무사사구, 5탈삼진, 4실점(4자책), 투구수 99개, 평균자책 7.20']}","{'박건우(3번타자)': ['4타수, 2안타(0홈런), 0볼넷, 1삼진, 1타점, 1득점, 시즌타율 0.500']}"
5,4.2 (일),14:00,한화,6,키움,7,고척,SPOTV2,"{'김재웅(승)': ['1이닝, 무피안타, 무사사구, 3탈삼진, 무실점(무자책), 투구수12개, 평균자책0.00']}","{'주현상(패)': ['0이닝, 2피안타, 2사사구, 0탈삼진, 1실점(1자책), 투구수 19개, 평균자책 9.00']}","{'김휘집(7번타자)': ['0타수, 0안타(0홈런), 1볼넷, 0삼진, 1타점, 1득점, 시즌타율 0.000']}"
6,4.2 (일),14:00,KIA,9,SSG,5,문학,SBS SPORTS,"{'이의리(승)': ['5이닝, 3피안타, 6사사구, 3탈삼진, 3실점(1자책), 투구수101개, 평균자책1.80']}","{'맥카티(패)': ['3 ⅓이닝, 1무피안타, 3사사구, 1탈삼진, 8실점(8자책), 투구수 90개, 평균자책 21.60']}","{'최형우(9번타자)': ['1타수, 1안타(0홈런), 0볼넷, 0삼진, 2타점, 0득점, 시즌타율 0.200']}"
7,4.2 (일),14:00,LG,10,KT,9,수원,SPOTV,"{'함덕주(승)': ['2이닝, 무피안타, 무사사구, 4탈삼진, 무실점(무자책), 투구수29개, 평균자책0.00']}","{'박영현(패)': ['0 ⅔이닝, 2피안타, 무사사구, 1탈삼진, 1실점(1자책), 투구수 11개, 평균자책 6.75']}","{'이천웅(9번타자)': ['1타수, 1안타(0홈런), 0볼넷, 0삼진, 1타점, 0득점, 시즌타율 1.000']}"
8,4.2 (일),14:00,롯데,2,두산,0,잠실,KBS N SPORTS,"{'나균안(승)': ['6 ⅔이닝, 5피안타, 2사사구, 4탈삼진, 무실점(무자책), 투구수100개, 평균자책0.00']}","{'최원준(패)': ['7이닝, 5피안타, 1사사구, 2탈삼진, 2실점(2자책), 투구수 96개, 평균자책 2.57']}","{'한동희(6번타자)': ['4타수, 1안타(0홈런), 0볼넷, 0삼진, 2타점, 0득점, 시즌타율 0.091']}"
9,4.2 (일),14:00,NC,6,삼성,8,대구,MBC SPORTS+,"{'이승현(승)': ['2이닝, 1피안타, 무사사구, 2탈삼진, 무실점(무자책), 투구수26개, 평균자책0.00']}","{'김진호(패)': ['1이닝, 2피안타, 무사사구, 1탈삼진, 2실점(2자책), 투구수 21개, 평균자책 9.00']}","{'김지찬(1번타자)': ['4타수, 1안타(0홈런), 1볼넷, 1삼진, 1타점, 2득점, 시즌타율 0.375']}"


In [ ]:
fetch("https://api-gw.sports.naver.com/schedule/games/20230602SSHH02023/record", {
  "headers": {
    "accept": "application/json, text/plain, */*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "sec-ch-ua": "\"Whale\";v=\"3\", \"Not-A.Brand\";v=\"8\", \"Chromium\";v=\"114\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "Referer": "https://m.sports.naver.com/game/20230602SSHH02023/record",
    "Referrer-Policy": "unsafe-url"
  },
  "body": null,
  "method": "GET"
});

In [4]:
list(pit_record.values())[1]

[{'bb': 2,
  'kk': 4,
  'seasonLose': 0,
  'ab': 25,
  'gameCount': 3,
  'bf': 98,
  'seasonWin': 0,
  'pcode': '52606',
  'bbhp': 2,
  'inn': '6 ⅔',
  'hr': 0,
  'l': 0,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 27,
  'hit': 5,
  'r': 0,
  's': 0,
  'era': '1.15',
  'w': 1,
  'name': '파노니',
  'wls': '승'},
 {'bb': 0,
  'kk': 0,
  'seasonLose': 0,
  'ab': 1,
  'gameCount': 38,
  'bf': 2,
  'seasonWin': 0,
  'pcode': '62754',
  'bbhp': 0,
  'inn': '0 ⅓',
  'hr': 0,
  'l': 1,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 1,
  'hit': 0,
  'r': 0,
  's': 2,
  'era': '2.59',
  'w': 1,
  'name': '임기영',
  'wls': '홀'},
 {'bb': 1,
  'kk': 0,
  'seasonLose': 0,
  'ab': 5,
  'gameCount': 30,
  'bf': 21,
  'seasonWin': 0,
  'pcode': '63950',
  'bbhp': 1,
  'inn': '0 ⅔',
  'hr': 1,
  'l': 2,
  'er': 2,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 6,
  'hit': 3,
  'r': 2,
  's': 3,
  'era': '4.00',
  'w': 1,
  'name': '장현식',
  'wls': ''},
 {'bb': 0,
  'kk': 0,
  'seasonLose': 

In [44]:
[pit_info for pit_info in list(pit_record.values())[0] and list(pit_record.values())[1] if pit_info["wls"] == "승"]

[{'bb': 0,
  'kk': 0,
  'seasonLose': 0,
  'ab': 3,
  'gameCount': 1,
  'bf': 14,
  'seasonWin': 0,
  'pcode': '60336',
  'bbhp': 0,
  'inn': '1',
  'hr': 0,
  'l': 0,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 3,
  'hit': 0,
  'r': 0,
  's': 0,
  'era': '0.00',
  'w': 1,
  'name': '문성현',
  'wls': '승'}]

In [21]:
game_id = raw_record_url.split('/')[2]

In [27]:
record_url='https://api-gw.sports.naver.com/schedule'+raw_record_url.replace('game', 'games')
record_rp=requests.get(record_url)
json.loads(record_rp.text)

{'code': 200,
 'success': True,
 'result': {'recordData': {'etcRecords': [{'result': '이형종(10회 2사 만루서 좌전 안타)',
     'how': '결승타'},
    {'result': '문현빈(7회) \r\n\r\n', 'how': '3루타'},
    {'result': '러셀(3회) 노시환(8회) 김혜성(10회) \r\n\r\n', 'how': '2루타'},
    {'result': '박정현(4회) 송성문(7회) 이지영(8회) 김태진(8회) \r\n\r\n', 'how': '실책'},
    {'result': '문현빈(7회) 정은원(9회) \r\n\r\n', 'how': '주루사'},
    {'result': '이형종(8회) \r\n\r\n', 'how': '병살타'},
    {'result': '안우진(6회) \r\n\r\n', 'how': '폭투'},
    {'result': '박종철 오훈규 문동균 나광남', 'how': '심판'}],
   'homeTeamNextGames': [{'gameId': '20230402HHWO02023',
     'aName': '한화',
     'gmkey': '20230402HHWO0',
     'gyear': 2023,
     'gweek': '일',
     'aCode': 'HH',
     'stadium': '고척',
     'hCode': 'WO',
     'hName': '키움',
     'gdate': 20230402},
    {'gameId': '20230404LGWO02023',
     'aName': 'LG',
     'gmkey': '20230404LGWO0',
     'gyear': 2023,
     'gweek': '화',
     'aCode': 'LG',
     'stadium': '고척',
     'hCode': 'WO',
     'hName': '키움',
     'gdate':

In [23]:
'https://api-gw.sports.naver.com/schedule/games/20230602SSHH02023/record'

'https://api-gw.sports.naver.com/schedule/games/20230602SSHH02023/record'

In [13]:
record_url

'https://api-gw.sports.naver.com/schedule/game/20230401HHWO02023/record'

In [5]:
tr.select('td>strong>em.vs')

[]

In [41]:
[pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']


[{'bb': 0,
  'kk': 0,
  'seasonLose': 0,
  'ab': 3,
  'gameCount': 1,
  'bf': 14,
  'seasonWin': 0,
  'pcode': '60336',
  'bbhp': 0,
  'inn': '1',
  'hr': 0,
  'l': 0,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 3,
  'hit': 0,
  'r': 0,
  's': 0,
  'era': '0.00',
  'w': 1,
  'name': '문성현',
  'wls': '승'}]

In [30]:
list(pit_info)

[{'bb': 2,
  'kk': 12,
  'seasonLose': 0,
  'ab': 23,
  'gameCount': 1,
  'bf': 112,
  'seasonWin': 0,
  'pcode': '68341',
  'bbhp': 2,
  'inn': '6',
  'hr': 0,
  'l': 0,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 25,
  'hit': 5,
  'r': 0,
  's': 0,
  'era': '0.00',
  'w': 0,
  'name': '안우진',
  'wls': ''},
 {'bb': 1,
  'kk': 1,
  'seasonLose': 0,
  'ab': 5,
  'gameCount': 1,
  'bf': 25,
  'seasonWin': 0,
  'pcode': '62360',
  'bbhp': 1,
  'inn': '1',
  'hr': 0,
  'l': 0,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 6,
  'hit': 1,
  'r': 1,
  's': 0,
  'era': '0.00',
  'w': 0,
  'name': '김태훈',
  'wls': '홀'},
 {'bb': 0,
  'kk': 1,
  'seasonLose': 0,
  'ab': 5,
  'gameCount': 1,
  'bf': 15,
  'seasonWin': 0,
  'pcode': '76118',
  'bbhp': 0,
  'inn': '1',
  'hr': 0,
  'l': 0,
  'er': 0,
  'hasPlayerEnd': True,
  'tb': 'B',
  'pa': 5,
  'hit': 1,
  'r': 1,
  's': 0,
  'era': '0.00',
  'w': 0,
  'name': '원종현',
  'wls': ''},
 {'bb': 0,
  'kk': 0,
  'seasonLose': 0,
  '

In [49]:
win_pit_info

''